In [45]:
import os
import pandas as pd
import requests
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_columns', 30)

- [ ] get the data from S3
- [ ] weather data transformations
- [x] taxi trips transformations
- [x] update payment type master
- [x] update company master
- [x] update taxi trips with company and payment type ids (replace the string values with ids from the latest master tables)
- [ ] upload weather data to s3
- [ ] upload taxi data to s3
- [ ] upload the newest payment type master and company master

### taxi trips transformation codes

In [2]:
current_datetime = datetime.now() - relativedelta(months=8)
formatted_datetime = current_datetime.strftime('%Y-%m-%d')

url = f"https://data.cityofchicago.org/resource/wrvz-psew.json?$where=trip_start_timestamp >= '{formatted_datetime}T00:00:00' AND trip_start_timestamp <= '{formatted_datetime}T23:59:59'&$limit=30000"

headers = {'X-App-Token': os.environ.get('CHICAGO_API_TOKEN')}

response = requests.get(url, headers)

data = response.json()

In [3]:
taxi_trips = pd.DataFrame(data)

In [4]:
taxi_trips.drop(['pickup_census_tract', 'dropoff_census_tract'], axis=1, inplace=True)
taxi_trips.drop(['pickup_centroid_location', 'dropoff_centroid_location'], axis=1, inplace=True)

taxi_trips.dropna(inplace=True)

taxi_trips.rename(
    columns={
        'pickup_community_area': 'pickup_community_area_id',
        'dropoff_community_area': 'dropoff_community_area_id'
    },
    inplace=True
)

taxi_trips['datetime_for_weather'] = pd.to_datetime(taxi_trips['trip_start_timestamp']).dt.floor('h')

In [5]:
taxi_trips.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather
0,95f5fcf8751fbd12ee73b61c45237f5125cf955f,b16774f4cc26db3c0bb6beb35343dd596516566be79316...,2023-12-12T23:45:00.000,2023-12-13T00:00:00.000,1048,0,32,22,17.92,0,0,0,17.92,Mobile,5 Star Taxi,41.878865584,-87.625192142,41.92276062,-87.699155343,2023-12-12 23:00:00
1,8ee8599fa81e304faaac145aa6338af53dfa3894,b5d74b92d646ed1edaff1b165977d2a4a5d06879202484...,2023-12-12T23:45:00.000,2023-12-13T00:00:00.000,588,5.51,76,10,15.75,0,0,5,20.75,Cash,Sun Taxi,41.980264315,-87.913624596,41.985015101,-87.804532006,2023-12-12 23:00:00
2,8b312a0cfac7b7e917325318fedd901451122e99,b5ad9d970e0745256fbb7517babae43abcb43115d076df...,2023-12-12T23:45:00.000,2023-12-13T00:00:00.000,720,10.7,24,69,27.5,0,0,0,27.5,Unknown,Taxi Affiliation Services,41.901206994,-87.676355989,41.763246799,-87.616134111,2023-12-12 23:00:00
3,83964fbad69255d149823875c5091a6354865738,3618045f9110d4d88482266ade23659c1a50d32ac37f20...,2023-12-12T23:45:00.000,2023-12-12T23:45:00.000,540,4.8,35,28,14.5,0,0,0,14.5,Unknown,Taxi Affiliation Services,41.835117986,-87.618677767,41.874005383,-87.66351755,2023-12-12 23:00:00
4,8236c10bdab73bbcf813cc952b6892c4be53103e,5cdde36a39ded2651da1686c7813baf589dac6eb873894...,2023-12-12T23:45:00.000,2023-12-13T00:00:00.000,600,0.1,28,8,9,5,0,0.5,14.5,Credit Card,Taxi Affiliation Services,41.874005383,-87.66351755,41.899602111,-87.633308037,2023-12-12 23:00:00


#### taxi trips transformation function

In [6]:
def taxi_trips_transformations(taxi_trips: pd.DataFrame) -> pd.DataFrame:

    """Perform transformations with the taxi data.

    Parameters
    ----------
    taxi_trips: pd.DataFrame
        The DataFrame holding the daily taxi trips.

    Returns
    -------
    pd.DataFrame
        The cleaned, transformed DataFrame holding the daily taxi trips.
    """

    if not isinstance(taxi_trips, pd.DataFrame):
        raise TypeError('taxi_trips is not a valid pandas DataFrame.')
    
    taxi_trips.drop(['pickup_census_tract', 'dropoff_census_tract', 'pickup_centroid_location', 'dropoff_centroid_location'], axis=1, inplace=True)

    taxi_trips.dropna(inplace=True)

    taxi_trips.rename(
        columns={
            'pickup_community_area': 'pickup_community_area_id',
            'dropoff_community_area': 'dropoff_community_area_id'
        },
        inplace=True
    )
    
    taxi_trips['datetime_for_weather'] = pd.to_datetime(taxi_trips['trip_start_timestamp']).dt.floor('h')

    return taxi_trips

#### company update codes

In [7]:
company_master = taxi_trips['company'].drop_duplicates().reset_index(drop=True)

company_master = pd.DataFrame(
    {
        'company_id': range(1, len(company_master) + 1),
        'company': company_master
    }
)

company_master.tail()

,company_id,company
25,26,6574 - Babylon Express Inc.
26,27,Taxi Affiliation Services Llc - Yell
27,28,3556 - 36214 RC Andrews Cab
28,29,5167 - 71969 5167 Taxi Inc
29,30,Petani Cab Corp


In [8]:
new_company_data = [
    {'company': '6574 - Babylon Express Inc.'},
    {'company': 'X'},
    {'company': 'Y'}
]

new_company_mapping = pd.DataFrame(new_company_data)

new_company_mapping

,company
0,6574 - Babylon Express Inc.
1,X
2,Y


In [9]:
company_max_id = company_master['company_id'].max()
company_max_id

30

In [10]:
# new_companies_list = []

# for company in new_company_mapping['company'].values:
#     if company not in company_master['company'].values:
#         new_companies_list.append(company)

# one line

new_companies_list = [company for company in new_company_mapping['company'].values if company not in company_master['company'].values]

new_companies_list

['X', 'Y']

In [11]:
new_companies_df = pd.DataFrame(
    {
        'company_id': range(company_max_id + 1, company_max_id + len(new_companies_list) + 1),
        'company': new_companies_list
    }
)

new_companies_df

,company_id,company
0,31,X
1,32,Y


In [12]:
updated_company_master = pd.concat([company_master, new_companies_df], ignore_index=True)

updated_company_master.tail()

,company_id,company
27,28,3556 - 36214 RC Andrews Cab
28,29,5167 - 71969 5167 Taxi Inc
29,30,Petani Cab Corp
30,31,X
31,32,Y


In [13]:
def update_company_master(taxi_trips: pd.DataFrame, company_master: pd.DataFrame) -> pd.DataFrame:
    
    """Extend the company master with new companies if there are new companies.

    Parameters
    ----------
    taxi_trips: pd.DataFrame
        DataFrame holding the daily taxi trips.
    company_master:
        DataFrame holding the company_master data.

    Returns
    -------
    pd.DataFrame
        The updated company_master, if new companies are in the taxi data, they will be loaded to it.
    """
    
    company_max_id = company_master['company_id'].max()
    
    new_companies_list = [company for company in taxi_trips['company'].values if company not in company_master['company'].values]

    new_companies_df = pd.DataFrame(
        {
            'company_id': range(company_max_id + 1, company_max_id + len(new_companies_list) + 1),
            'company': new_companies_list
        }
    )

    updated_company_master = pd.concat([company_master, new_companies_df], ignore_index=True)

    return updated_company_master

In [14]:
taxi_trips_company_only = pd.DataFrame(
    {
        'company_id': [1,2,3],
        'company': ['6574 - Babylon Express Inc.', 'X', 'Y']
    }
)

taxi_trips_company_only

,company_id,company
0,1,6574 - Babylon Express Inc.
1,2,X
2,3,Y


In [15]:
updated_company_master = update_company_master(taxi_trips=taxi_trips_company_only, company_master=company_master)

In [16]:
updated_company_master.tail()

,company_id,company
27,28,3556 - 36214 RC Andrews Cab
28,29,5167 - 71969 5167 Taxi Inc
29,30,Petani Cab Corp
30,31,X
31,32,Y


#### payment type master codes

In [17]:
payment_type_master = taxi_trips['payment_type'].drop_duplicates().reset_index(drop=True)

payment_type_master = pd.DataFrame(
    {
        'payment_type_id': range(1, len(payment_type_master) + 1),
        'payment_type': payment_type_master
    }
)

taxi_trips_payment_type_only = pd.DataFrame(
    {
        'payment_type_id': [1,2,3],
        'payment_type': ['Credit Card', 'X', 'Y']
    }
)

taxi_trips_payment_type_only

,payment_type_id,payment_type
0,1,Credit Card
1,2,X
2,3,Y


In [18]:
def update_payment_type_master(taxi_trips: pd.DataFrame, payment_type_master: pd.DataFrame) -> pd.DataFrame:
    
    """Extend the payment type master with new payment types if there are new payment types.

    Parameters
    ----------
    taxi_trips: pd.DataFrame
        DataFrame holding the daily taxi trips.
    payment_type_master:
        DataFrame holding the payment type master data.

    Returns
    -------
    pd.DataFrame
        The updated payment type master data, if new payment types are in the taxi data, they will be loaded to it.
    """
    
    payment_type_max_id = payment_type_master['payment_type_id'].max()
    
    new_payment_types_list = [payment_type for payment_type in taxi_trips['payment_type'].values if payment_type not in payment_type_master['payment_type'].values]

    new_payment_type_df = pd.DataFrame(
        {
            'payment_type_id': range(payment_type_max_id + 1, payment_type_max_id + len(new_payment_types_list) + 1),
            'payment_type': new_payment_types_list
        }
    )

    updated_payment_type_master = pd.concat([payment_type_master, new_payment_type_df], ignore_index=True)

    return updated_payment_type_master

In [19]:
updated_payment_type_master = update_payment_type_master(taxi_trips=taxi_trips_payment_type_only, payment_type_master=payment_type_master)

In [20]:
updated_payment_type_master

,payment_type_id,payment_type
0,1,Mobile
1,2,Cash
2,3,Unknown
3,4,Credit Card
4,5,Prcard
5,6,Dispute
6,7,No Charge
7,8,X
8,9,Y


#### Creating a generic update master table function

In [21]:
def update_master(taxi_trips: pd.DataFrame, master: pd.DataFrame, id_column: str, value_column: str) -> pd.DataFrame:
    
    """Extend the master DataFrame with new values if there are any.

    Parameters
    ----------
    taxi_trips: pd.DataFrame
        DataFrame holding the daily taxi trips.
    master: pd.DataFrame
        DataFrame holding the master data.
    id_column: str
        The id column of the master DataFrame.
    value_column : str
        Name of the column in master_df containing the values.

    Returns
    -------
    pd.DataFrame
        The updated master data, if new values are in the taxi data, they will be loaded to it.
    """
    
    max_id = master[id_column].max()
    
    new_values_list = [value for value in taxi_trips[value_column].values if value not in master[value_column].values]

    new_values_df = pd.DataFrame(
        {
            id_column: range(max_id + 1, max_id + len(new_values_list) + 1),
            value_column: new_values_list
        }
    )

    updated_master = pd.concat([master, new_values_df], ignore_index=True)

    return updated_master

In [22]:
test_payment_type_master = update_master(
    taxi_trips=taxi_trips_payment_type_only, 
    master=payment_type_master, 
    id_column='payment_type_id', 
    value_column='payment_type'
)

In [23]:
test_payment_type_master

,payment_type_id,payment_type
0,1,Mobile
1,2,Cash
2,3,Unknown
3,4,Credit Card
4,5,Prcard
5,6,Dispute
6,7,No Charge
7,8,X
8,9,Y


In [24]:
test_company_master = update_master(
    taxi_trips=taxi_trips_company_only, 
    master=company_master, 
    id_column='company_id', 
    value_column='company'
)

In [25]:
test_company_master.tail()

,company_id,company
27,28,3556 - 36214 RC Andrews Cab
28,29,5167 - 71969 5167 Taxi Inc
29,30,Petani Cab Corp
30,31,X
31,32,Y


### update taxi trips with the most recent company master and payment type master function

In [40]:
def update_taxi_trips_with_master_data(taxi_trips: pd.DataFrame, payment_type_master: pd.DataFrame, company_master: pd.DataFrame) -> pd.DataFrame:
    
    """ Update the taxi trips DataFrame with the company master and payment type master ids and delete the string columns.

    Parameters
    ----------
    taxi_trips: pd.DataFrame
        The DataFrame with the daily taxi trips.
    payment_type_master: pd.DataFrame
        The payment type master table.
    company_master: pd.DataFrame
        The company master table.

    Returns
    -------
    pd.DataFrame
        The taxi trips data with only payment type id and company id without company or payment type values.
    """

    taxi_trips_id = taxi_trips.merge(payment_type_master, on='payment_type')
    taxi_trips_id = taxi_trips_id.merge(company_master, on='company')
    
    taxi_trips_id.drop(['payment_type', 'company'], axis=1, inplace=True)

    return taxi_trips_id

In [42]:
taxi_trips_id = update_taxi_trips_with_master_data(taxi_trips=taxi_trips, payment_type_master=payment_type_master, company_master=company_master)

taxi_trips_id.sample(5)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather,payment_type_id,company_id
7347,d615471ab4e65a482b258dc4bce956b9c13b3011,db0763db8da1aee92628f7b0111bf1a6a8db4c8d06eb5c...,2023-12-12T15:15:00.000,2023-12-12T15:45:00.000,1768,5.69,8,22,20.25,4.15,0,0,24.9,41.899602111,-87.633308037,41.92276062,-87.699155343,2023-12-12 15:00:00,4,2
1803,1f07d9b9681b202a184d448f2b18bea10a504b19,9de7f6aa5035ab0e38c4e42f261923cec4b3148c4fa298...,2023-12-12T20:15:00.000,2023-12-12T20:30:00.000,1080,11.6,8,13,30,0,0,0,30,41.899602111,-87.633308037,41.983636307,-87.723583185,2023-12-12 20:00:00,3,3
17177,6cf4d8a7b3733d85bfbdd1a1108c5738f1824a69,9a423881d4046f5590943c08ec5e58d59f5ebd91fe26aa...,2023-12-12T00:45:00.000,2023-12-12T00:45:00.000,60,0,3,3,3.25,0,0,3.5,6.75,41.96581197,-87.655878786,41.96581197,-87.655878786,2023-12-12 00:00:00,2,3
16181,4fe3d09a8e8c1bae9679ea05cbaddab229989b0c,f7cc14447361ed368a0904327c2be78af8e9a858218741...,2023-12-12T07:30:00.000,2023-12-12T08:15:00.000,3341,8.97,28,28,32.5,0,0,0,32.5,41.874005383,-87.66351755,41.874005383,-87.66351755,2023-12-12 07:00:00,5,1
2355,8b6b0593a75fb53a4e527927cd7bebe75b7d61e2,4ea76937237d234148f31343797e28d4616f50891cb565...,2023-12-12T19:30:00.000,2023-12-12T19:30:00.000,496,1.31,8,28,7.5,0,0,0,7.5,41.892507781,-87.626214906,41.885300022,-87.642808466,2023-12-12 19:00:00,2,2


### weather transformations function

In [50]:
def transform_weather_data(weather_data: json) -> pd.DataFrame:
    
    """ Make transformations on the daily weather api response.

    Parameters
    ----------
    weather_data: json
        The daily weather data from the Open Meteo API.

    Returns
    -------
    pd.DataFrame
        A DataFrame representation of the data.
    """
    
    weather_data_filtered = {
        'datetime': weather_data['hourly']['time'],
        'temperature': weather_data['hourly']['temperature_2m'],
        'wind_speed': weather_data['hourly']['wind_speed_10m'],
        'rain': weather_data['hourly']['rain'],
        'precipitation': weather_data['hourly']['precipitation'] 
    }
    
    weather_df = pd.DataFrame(weather_data_filtered)
    
    weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])

    return weather_df

In [51]:
current_datetime = datetime.now() - relativedelta(months=8)
formatted_datetime = current_datetime.strftime('%Y-%m-%d')

url = 'https://archive-api.open-meteo.com/v1/era5'

params = {
    'latitude': 41.85,
    'longitude': -87.65,
    'start_date': formatted_datetime,
    'end_date': formatted_datetime,
    'hourly': 'temperature_2m,wind_speed_10m,rain,precipitation'
}

response = requests.get(url, params=params)

weather_data = response.json()

weather_data_df = transform_weather_data(weather_data=weather_data)

In [54]:
weather_data_df.head()

,datetime,temperature,wind_speed,rain,precipitation
0,2023-12-12 00:00:00,-0.7,13.6,0.0,0.0
1,2023-12-12 01:00:00,-0.7,16.5,0.0,0.0
2,2023-12-12 02:00:00,-0.4,18.9,0.0,0.0
3,2023-12-12 03:00:00,-0.0,20.5,0.0,0.0
4,2023-12-12 04:00:00,0.1,21.6,0.0,0.0
